<a href="https://colab.research.google.com/github/duongnt0201/duongnt0201/blob/main/%5BBlueStars%5D_Auto_B%C3%A1o_C%C3%A1o_K%E1%BA%BF_To%C3%A1n_Daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách Thực Hiện Auto Làm Báo Cáo Kế Toán**
*   B1: Tải các báo cáo SP, SD, SB về chung một file excel, để riêng ở từng sheet

*   B2: Đổi tên Sheet: Đổi tên các Sheet thành "SP", "SD" và "SB". Ví dụ sheet1 chứa data của SP thì đổi thành "SP"

*   B3: Lưu file lại, và bấm tổ hợp phím **Ctrl + F9**, hoặc bấm **Runtime** trên thanh công cụ của file này chọn **Run all**

***Lưu Ý***:
*   Đoạn code sẽ tự động chạy và tải về kết quả, nếu không có kết quả tải về thì đoạn code đã bị lỗi, cần check lại các bước đã làm đúng hay chưa. Thời gian chạy code khoảng 20s

*   Nếu chẳng may xóa một đoạn code nào đó, reload lại trang này để khôi phục code

*   Phải đăng nhập google drive mới có thể chạy Run all

*   Setting ngôn ngữ trên Google Sheet phải chuyển sang ***Hoa Kỳ*** thì khi paste từ excel lên mới không bị lỗi format



In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import io

In [ ]:
uploaded = files.upload()

Saving Raw data CA Daily.xlsx to Raw data CA Daily.xlsx


In [ ]:
dfs = {}

for file_name in uploaded.keys():
    xls = pd.ExcelFile(file_name)
    sheet_names = xls.sheet_names

    for sheet_name in sheet_names:
        dfs[sheet_name] = pd.read_excel(file_name, sheet_name=sheet_name)

SP = dfs['Sponsored_Products_Advertised_p']
SD = dfs['Sponsored_Display_Advertised_pr']
SB = dfs['Sheet2']

In [ ]:
SP_data = SP.groupby(['Date', 'Advertised SKU']).agg({'Spend':'sum', '7 Day Total Sales ':'sum'})
SP_data.rename(columns={'Advertised SKU':'SKU', "Spend":"Spend", "7 Day Total Sales ":"Sales"},inplace = True)
SP_data = SP_data.reset_index()

In [ ]:
SD_data = SD.groupby(['Date', 'Advertised SKU']).agg({'Spend':'sum', '14 Day Total Sales ':'sum'})
SD_data.rename(columns={'Advertised SKU':'SKU', "Spend":"Spend", "14 Day Total Sales ":"Sales"},inplace = True)
SD_data = SD_data.reset_index()

In [ ]:
product_id = pd.read_html('https://docs.google.com/spreadsheets/d/e/2PACX-1vQKItZA2bNZCY2yD52UEunCjkuq8e9yDuHLQzLqbOvLy13GeJWLWFmujTCRbrBVNA/pubhtml?gid=678803432&single=true', skiprows=1)[0]
product_id = product_id.iloc[1:, 1:]

In [ ]:
product_id['SKU'] = product_id['SKU'].astype(str)
def get_matching_sku(campaign_name):
    for sku in product_id['SKU']:
        # Ensure campaign_name is a string before the 'in' check
        if isinstance(campaign_name, str) and sku in campaign_name:
            return sku
    # Return None if no match is found or if campaign_name is not a string
    return None

# Convert the 'Campaign' column to string type, handling potential non-string values
SB['Campaign'] = SB['Campaign'].astype(str)

SB['Advertised SKU'] = SB['Campaign'].apply(get_matching_sku)

In [ ]:
campaigns_without_sku = SB.loc[SB['Advertised SKU'] ==  'None','Campaign']

unique_campaigns_without_sku = campaigns_without_sku.drop_duplicates().tolist()

if unique_campaigns_without_sku:
    print("Các campaigns sau chưa có SKU trong danh sách sản phẩm:")
    print(unique_campaigns_without_sku)
else:
    print("Dữ liệu hoàn chỉnh")

Dữ liệu hoàn chỉnh


In [ ]:
# SB_data['Date'] = pd.to_datetime(SB_data['Date'], format='%Y%m%d').dt.strftime('%m/%d/%Y')
SB_data = SB.groupby(['Date', 'Advertised SKU']).agg({'Spend':'sum', 'Sales':'sum'})
SB_data.rename(columns={'Advertised SKU':'SKU', "Spend":"Spend", "Sales":"Sales"},inplace = True)
SB_data = SB_data.reset_index()

In [ ]:
data = pd.concat([SP_data, SD_data, SB_data])
data = data.reset_index()
data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%m/%d/%Y')

In [ ]:
# Remove non-numeric characters from 'Sales' column and convert to float
data['Sales'] = data['Sales'].astype(str).str.replace(r'[^\d.]', '', regex=True)
data['Sales'] = pd.to_numeric(data['Sales'], errors='coerce')

# Remove non-numeric characters from 'Spend' column and convert to float
data['Spend'] = data['Spend'].astype(str).str.replace(r'[^\d.]', '', regex=True)
data['Spend'] = pd.to_numeric(data['Spend'], errors='coerce')


final_data = data.groupby(['Date','Advertised SKU']).agg({'Spend':'sum', 'Sales':'sum'})
final_data = final_data.astype({'Spend':'float', 'Sales':'float'})
final_data = final_data.reset_index()

In [ ]:
# Assume store name is in the 'Country' column; get the first unique value or use any specific logic
store = SB['Country'].iloc[0]  # Adjust this if you need to filter or select a specific row

# Date
date = (pd.Timestamp.now() - pd.Timedelta(days=1)).strftime('%Y.%m.%d')  #  date

# Create dynamic file name
file_name = f"{store} Daily {date}.xlsx"

# Export to Excel with dynamic file name
final_data.to_excel(file_name, index=False)

# Download the file
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>